# Finding Alternating Bit Strings Using Grover's Search Algorithm 

In this example we explore Grover's search algorithm, one of the most famous quantum algorithms. Grover's algorithm is an algorithm for solving the following search problem. Given a function $f$ which takes an N-bit input and returns a 1-bit output, 0 or 1, find any input $x_0$ for which $f(x_0) = 1$.

We can represent this problem on a quantum computer by defining an oracle $G$ that acts on our state as follows:
$$
G|x\rangle =  \begin{cases} 
      -|x\rangle & \text{if } |x\rangle = \text{a state for which }f(x) = 1\\
       |x\rangle & \text{otherwise}
   \end{cases}\\
$$

Grover's algorithm allows us to find a value $x$ for which $f(x) = 1$ with only $O(\sqrt{\frac{2^N}{k}})$ applications of $G$, where $k$ is the number of $x$'s for which $f(x) = 1$.  

#### Run these cells before beginning to import necessary packages.
This cell may take a minute or more to complete. Importing qsharp triggers a sequence of actions which allow us to compile and run Q# code in this notebook.

In [ ]:
import qsharp
from qsharp import azure
import matplotlib.pyplot as plt

In [ ]:
%%qsharp 
open Microsoft.Quantum.Diagnostics; 
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Random;

## Write the code

First we will define an oracle. In order to define an oracle, we need to know how to implement the function $f$. Grover's algorithm is suitable for problems in which defining $f$ is easy, while finding a value $x$ which satisfies the equation $f(x) = 1$ is not. One example would be a Sudoku puzzle, where it is easier to check whether a grid filled with digits is an answer than it is to find the answer.

For this example we will consider the function $f(x)$ which is equal to 1 when $x$ is a bit string of length $N$ in which no pair of adjacent bits have the same value. If you think about this, it becomes clear that the solutions to $f(x) = 1$ are bit strings which alternate between 0 and 1. This means that for any $N$ there will be two possible solutions, one starting with 0 and one starting with 1, so the number of solutions $k = 2$.

Our $f$ will look like this:
$$
f(x) =  \begin{cases} 
      1 & x = 0101\text{... or }1010\text{...}\\
      0 & \text{otherwise}
   \end{cases}\\
$$
However, our oracle will not mark these two states explicitly based on our ability to find the solution classically. Instead, it will use binary logic to check the condition by verifying that each pair of adjacent bits has different values. In this way we can build our oracle without actually knowing what the target states will be.

In [ ]:
%%qsharp
/// # Summary
/// A phase oracle which applies a negative phase to 
/// the states where no adjacent bits are equal.
///
/// # Input 
/// ## qs 
/// The qubits the oracle is applied to. 

operation AlternatingBitStringOracle(qs : Qubit[]) : Unit { 
    // We first apply a CNOT gate between each pair of adjacent qubits
    // to calculate qs[i] XOR qs[i-1] and store the result in qs[i-1].
    // Next, we apply a CZ to the output of all of these XOR operations
    // flipping the phase of the state where each XOR evaluates to 1.
    // Then we uncompute the previous CNOT gates to return the phase
    // to the target states. 

    // The within-apply statement does 3 things:
    // 1. Performs the operations in the within block.
    // 2. Performs the operations in the apply block.
    // 3. Calculates and performs the adjoint of the within block.
    within {
        for i in 1 .. Length(qs) - 1 {
            CNOT(qs[i], qs[i - 1]);
        }
    } apply {
        Controlled Z(qs[0 .. Length(qs) - 3], qs[Length(qs) - 2]);
    }
}

Next, we need to define Grover's diffusion operator. A lot has been written about the reasoning behind the diffusion operator; [this article](https://cnot.io/quantum_algorithms/grover/grovers_algorithm.html) does a great job of explaining it.



In [ ]:
%%qsharp
/// # Summary
/// Grover's diffusion operator.
///
/// # Input
/// ## qs
/// The qubit array the diffusion operator is applied to.

operation Diffusion(qs : Qubit[]) : Unit {
    within {
        ApplyToEachA(H, qs);
        ApplyToEachA(X, qs);
    } apply {
        // Most gives us every array element except the last.
        // Tail gives us the last array element.
        Controlled Z(Most(qs), Tail(qs));
    }
}

Lastly, let us define our top-level operation. In it we will allocate the qubits we will use, apply the operations `AlternatingBitStringOracle` and `Diffusion` a specified number of times, and measure the qubits to read out the solution. Note that instead of calculating the optimal number of iterations during runtime we set things up so we can control the number when calling the operation.

In [ ]:
%%qsharp
/// # Summary
/// The top-level operation for this implementation. 
/// Applies the oracle and the diffusion operator multiple times.
///
/// # Input
/// ## iterations
/// The number of times to apply the oracle and the diffusion operator.
/// ## N 
/// The number of bits in the bit strings. Our search space size is then 2^N.
///
/// # Output
/// A Result[] type which contains the measurement result of each
/// qubit that was allocated. 

operation Grovers(iterations : Int, N : Int) : Result[] {
    use qs = Qubit[N];
    ApplyToEach(H, qs);

    for i in 1 .. iterations {
        AlternatingBitStringOracle(qs);
        Diffusion(qs);
    }
    return ForEach(M, qs);
}

## Run local simulation

Now you can run this cell to see whether our measurement produced a state that is a solution to our problem. 

In [ ]:
Grovers.simulate(iterations=1, N=3)

## Run on Azure Quantum cloud targets

Connect to Azure workspace.

In [ ]:
qsharp.azure.connect(
   resourceId="",
   location="")

Set the target to ionq.simulator, ionq.qpu, or ionq.qpu-aria-1, depending on which target you want to use.

In [ ]:
qsharp.azure.target("ionq.simulator")

In [ ]:
N = 3
iterations = 1
qsharp.azure.submit(Grovers, iterations=iterations, N=N, shots=500, jobName="Grover's iterations={}, N={}".format(iterations, N))

In [ ]:
qsharp.azure.status()

In [ ]:
output = qsharp.azure.output(jobId="")

In [ ]:
def plot_job_results(id, iter, sim):
    output = qsharp.azure.output(id)
    print(output)
    keys = list(output.keys())
    keys = [key.replace(",", "") for key in keys]
    keyLen = len(keys[0]) - 2
    firstBitString = ""
    secondBitString = ""
    for i in range(keyLen):
        firstBitString += str((i+1)%2)
        secondBitString += str(i%2)
    secondBitString = "[{}]".format(secondBitString)
    firstBitString = "[{}]".format(firstBitString)
    outputFreq = list(output.values())

    firstBitStringLoc = min(keys.index(firstBitString), keys.index(secondBitString))
    secondBitStringLoc = max(keys.index(firstBitString), keys.index(secondBitString))

    plt.bar(keys[0:firstBitStringLoc], outputFreq[0:firstBitStringLoc], color = "blue")
    plt.bar(keys[firstBitStringLoc], outputFreq[firstBitStringLoc], color = "red")
    plt.bar(keys[firstBitStringLoc+1:secondBitStringLoc], outputFreq[firstBitStringLoc+1:secondBitStringLoc], color = "blue")
    plt.bar(keys[secondBitStringLoc], outputFreq[secondBitStringLoc], color = "red")
    plt.bar(keys[secondBitStringLoc+1:], outputFreq[secondBitStringLoc+1:], color = "blue")
    plt.xticks(rotation=90)
    plt.title("{} results for N = {} iter = {}".format(sim, keyLen, iter))
    plt.show()

In [ ]:
plot_job_results("", iterations, "(target name)")

> `qsharp.azure.output` defaults to fetching the results of the last submitted job. 
We recommend submitting all hardware jobs as soon as you can, since it can take a while for them to finish. 
You can then get the output for each of them by calling `qsharp.azure.output(jobId="job id")`. 
The job ids can be found in your Azure Quantum Workspace in Operations $\rightarrow$ Job management.